## Obtain the ambient temperature for the area at or near the earthquake epicenter.

The goal is to get temperature for 90 days prior to the earthquake, and get the temperature data for the same period for the previous 4 years for a total of 5 years.

In order to accomplish this, I have to find out if the temperature information is available for the complete time frame that I want.  This is dependent on the source of the weather data that I am using (DarkSky).  There is information out there but there is a reason services such as DarkSky exist.  At this point I have made the decision to work with the data that I can get right now in order to have a complete project.  If I get promising results, then I would look for a way to obtain more complete weather data.  

**At this point I am starting to look for an alternative to darksky, some of the data
    is not available, or it is spotty.**

In [1]:
using HTTP
using Dates
using CSV
using DataFrames
using Geodesy
using JSON
using DataStructures

In [2]:
base_url = "https://api.darksky.net/forecast"
earthquakes_csv = "../earthquakes_csv/1999_2017_earthquakes_magnitude_7-9.csv" #magnitude 7-9
cities_csv = "../cities_csv/world_cities_data.csv"

"../cities_csv/world_cities_data.csv"

In [3]:
function read_apikey(apikey_file)
    key = open(apikey_file) do file
          readlines(file)
        end 
    return String(key[1])
end

read_apikey (generic function with 1 method)

In [4]:
apikey = read_apikey("darksky_api.txt")
println("Done reading apikey!")

Done reading apikey!


In [5]:
earthquakes_df = DataFrame(CSV.File(earthquakes_csv))

,Column1,datetime,latitude,longitude,depth,magnitude
,Int64,String,Float64,Float64,Float64,Float64
1,96,2000-01-08T16:47:20.580Z,-16.925,-174.248,183.4,7.2
2,571,2000-02-25T01:43:58.640Z,-19.528,173.818,33.0,7.1
3,891,2000-03-28T11:00:22.510Z,22.338,143.73,126.5,7.6
4,1149,2000-04-23T09:27:23.320Z,-28.307,-62.99,608.5,7.0
5,1297,2000-05-04T04:21:16.210Z,-1.105,123.573,26.0,7.6
6,1420,2000-05-12T18:43:18.120Z,-23.548,-66.452,225.0,7.2
7,1674,2000-06-04T16:28:26.170Z,-4.721,102.087,33.0,7.9
8,1997,2000-06-18T14:44:13.310Z,-13.802,97.453,10.0,7.9
9,2869,2000-08-06T07:27:12.900Z,28.856,139.556,394.8,7.4


In [66]:
function unix_time(earthquake_time)
    
    if string(last(earthquake_time)) == string("Z")
        eq_time = strip(earthquake_time, last(earthquake_time))
    else
        eq_time = earthquake_time
    end
    
    return round(Int, Dates.datetime2unix(DateTime(eq_time)))
end

unix_time (generic function with 1 method)

In [7]:
function darksky_api_call(url)
    try
        response = HTTP.get(url)
        return String(response.body)
        catch e
        return "Error occured: $e"
    end
end

darksky_api_call (generic function with 1 method)

In [8]:
function cities_data()
     return DataFrame(CSV.File(cities_csv))
end

cities_data (generic function with 1 method)

In [9]:
function forecast_weather(latitude, longitude)
    forecast_url = base_url * "/" * apikey * "/" * string(latitude) * "," * string(longitude)
    return darksky_api_call(forecast_url)
end

forecast_weather (generic function with 1 method)

In [10]:
function get_historical_weather(latitude, longitude, earthquake_datetime)
    if typeof(earthquake_datetime) != Int64 
        unix_timestamp = unix_time(earthquake_datetime)
    else
        unix_timestamp = earthquake_datetime
    end
    
    historical_temperature_url = base_url * "/" * apikey * "/" * string(latitude) * "," * string(longitude) * "," * string(unix_timestamp)
    return darksky_api_call(historical_temperature_url)
end

get_historical_weather (generic function with 1 method)

In [11]:
# This will return the nearest city lat, lon and the distance in kilometers
function find_nearest_city(lat, lon)
    distancearray = []
    cities_df = cities_data() # This will receive a table, with the city data
    
    for i in eachrow(cities_df)
       city_coords = LLA(i["latitude"],i["longitude"])
       point = LLA(lat, lon)
       push!(distancearray, distance(city_coords, point)/1000)
    end
    
    distance_kms, index = findmin(distancearray)
    #nearest_city_lat, nearest_city_lon = cities_df[index, 3:4] # this returns a dataframe row, keeping this line to remember two ways to accomplish this
    nearest_city_lat, nearest_city_lon = cities_df[index, ["latitude", "longitude"]] # this returns a dataframe row
    return nearest_city_lat, nearest_city_lon, round(distance_kms, digits=0)
        
end

find_nearest_city (generic function with 1 method)

In [12]:
# Find the closest city to the earthquake
function create_earthquakes_nearest_cities_df()
   # eq_cities = DataFrame(nearest_city_latitude = Float64[], nearest_city_longitude = Float64[], nearest_city_distance = Float64[])
    
    eq_cities_df = DataFrame(earthquake_time = String[],
                            earthquake_latitude = Float64[],
                            earthquake_longitude = Float64[],
                            earthquake_depth = Float64[],
                            earthquake_magnitude = Float64[],
                            nearest_city_latitude = Float64[],
                            nearest_city_longitude = Float64[],
                            nearest_city_distance = Float64[])
    
    for earthquake in eachrow(earthquakes_df)
        city_latitude, city_longitude, city_distance_kilometers = find_nearest_city(earthquake["latitude"], earthquake["longitude"])
        push!(eq_cities_df, (earthquake_time = earthquake["datetime"],
                            earthquake_latitude = earthquake["latitude"],
                            earthquake_longitude = earthquake["longitude"],
                            earthquake_depth = earthquake["depth"],
                            earthquake_magnitude = earthquake["magnitude"],
                            nearest_city_latitude = city_latitude,
                            nearest_city_longitude = city_longitude,
                            nearest_city_distance = city_distance_kilometers))
    end
    return eq_cities_df
end

create_earthquakes_nearest_cities_df (generic function with 1 method)

### Keeping the "yes" data should be the only option,  I chose to all keep the "no" in case I find an alternate weather datasource.
If I want the data for which the historical data is available, then I will just filter the dataframe.

In [13]:
# This will have to be merged with the eq79 data since the nearest_city is the best chance at historical weather data to be available
function create_earthquakes_cities_historical_weather_df()
    
    nearestcity_df = create_earthquakes_nearest_cities_df()
    
    weather = []
    for city in eachrow(nearestcity_df)
        apicall_response = get_historical_weather(city["nearest_city_latitude"],city["nearest_city_longitude"],city["earthquake_time"])
        h = JSON.parse(apicall_response)
        
        if haskey(h["currently"], "apparentTemperature")
            push!(weather, "yes")
        else
            push!(weather, "no")
        end
    end
    insertcols!(nearestcity_df, :historical_temp_available => weather)
    return nearestcity_df # return table with weather availability
        
end

create_earthquakes_cities_historical_weather_df (generic function with 1 method)

In [14]:
earthquakes_cities_historical_weather_df = create_earthquakes_cities_historical_weather_df()

,earthquake_time,earthquake_latitude,earthquake_longitude,earthquake_depth
,String,Float64,Float64,Float64
1,2000-01-08T16:47:20.580Z,-16.925,-174.248,183.4
2,2000-02-25T01:43:58.640Z,-19.528,173.818,33.0
3,2000-03-28T11:00:22.510Z,22.338,143.73,126.5
4,2000-04-23T09:27:23.320Z,-28.307,-62.99,608.5
5,2000-05-04T04:21:16.210Z,-1.105,123.573,26.0
6,2000-05-12T18:43:18.120Z,-23.548,-66.452,225.0
7,2000-06-04T16:28:26.170Z,-4.721,102.087,33.0
8,2000-06-18T14:44:13.310Z,-13.802,97.453,10.0
9,2000-08-06T07:27:12.900Z,28.856,139.556,394.8


## Save the dataframe

In [15]:
#CSV.write("../earthquakes_csv/earthquakes_cities_historical_weather_df.csv", earthquakes_cities_historical_weather_df)

## Let's see for how many earthquakes with a magnitude 7-9, is the historical data available.

The results are not very promissing, because that means that for the chosen magnitudes about 40 percent have weather data available.  The weather data for some of those the data will probably not be good because of how far the measurements (temp) from the earthquake epicenter.

In [16]:
counter(earthquakes_cities_historical_weather_df.historical_temp_available)

Accumulator{Any,Int64} with 2 entries:
  "yes" => 108
  "no"  => 155

## Reload the newly created dataframe 
(Just in case, I had to do this because I did an inplace modification of the orinal df and was left with an empty df)

In [17]:
#earthquakes_cities_historical_weather_df = DataFrame(CSV.File("../earthquakes_csv/earthquakes_cities_historical_weather_df.csv"))

**Let's start workig with the magnitude 7 or greater for which there is historical temperature data and it is from a distance of less than 300 kilometers**

Create the new dataframe, we will need.

In [18]:
available_temperature_df = filter(row -> (row.historical_temp_available == "yes" && row.nearest_city_distance <= 300), earthquakes_cities_historical_weather_df)

,earthquake_time,earthquake_latitude,earthquake_longitude,earthquake_depth
,String,Float64,Float64,Float64
1,2001-01-10T16:02:44.230Z,57.078,-153.211,33.0
2,2001-01-26T03:16:40.500Z,23.419,70.232,16.0
3,2001-10-12T15:02:16.840Z,12.686,144.98,37.0
4,2002-04-26T16:06:07.000Z,13.088,144.619,85.7
5,2002-11-03T22:12:41.518Z,63.5141,-147.453,4.2
6,2003-05-26T09:24:33.400Z,38.849,141.568,68.0
7,2003-09-25T19:50:06.360Z,41.815,143.91,27.0
8,2003-09-25T21:08:00.030Z,41.774,143.593,33.0
9,2003-09-27T11:33:25.080Z,50.038,87.813,16.0


In [57]:
# Function to generate the dates ~90 days, for that year and the previous years.
function generate_historical_dates(timestamp)
    date = DateTime(strip(timestamp, last(string(timestamp))))
    date_array = []
    for i in 0:4
        for j in 0:90
            push!(date_array, string(date - Day(j) - Year(i)))
        end
    end
    return date_array
end 

generate_historical_dates (generic function with 1 method)

In [20]:
# Need to test a couple of times for 5 years, 90 days of data for each.
# save all the historical data.  This will mark the Air temperature data complete.

## Lets look at the temperature data from the API response 

In [33]:
temperature_info = get_historical_weather(25.93, 128.425,"2010-02-26T20:31:26.970Z")
#temperature_info = get_historical_weather(4.102, 123.907, "2001-10-19T03:28:44.460Z")

"{\"latitude\":25.93,\"longitude\":128.425,\"timezone\":\"Asia/Tokyo\",\"currently\":{\"time\":1267216287,\"uvIndex\":0},\"hourly\":{\"data\":[{\"time\":1267207200,\"temperature\":72.49,\"apparentTemperature\":73.48,\"dewPoint\":68,\"humidity\":0.86,\"pressure\":1013,\"windSpeed\":14.96,\"windBearing\":240,\"uvIndex\":0,\"visibility\":6.216},{\"time\":1267210800,\"temperature\":72.49,\"apparentTemperature\":73.48,\"dewPoint\":68,\"humidity\":0.86,\"pressure\":1013,\"windSpeed\":14.96,\"windBearing\":240,\"uvIndex\":0,\"visibility\":6.216}]},\"flags\":{\"sources\":[\"cmc\",\"gfs\",\"icon\",\"isd\",\"madis\"],\"nearest-station\":15.565,\"units\":\"us\"},\"offset\":9}\n"

In [34]:
temperature_info_json = JSON.parse(temperature_info)

Dict{String,Any} with 7 entries:
  "latitude"  => 25.93
  "hourly"    => Dict{String,Any}("data"=>Any[Dict{String,Any}("visibility"=>6.…
  "flags"     => Dict{String,Any}("units"=>"us","sources"=>Any["cmc", "gfs", "i…
  "longitude" => 128.425
  "offset"    => 9
  "timezone"  => "Asia/Tokyo"
  "currently" => Dict{String,Any}("time"=>1267216287,"uvIndex"=>0)

In [35]:
temperature_info_json["hourly"]["data"]

2-element Array{Any,1}:
 Dict{String,Any}("visibility" => 6.216,"apparentTemperature" => 73.48,"time" => 1267207200,"pressure" => 1013,"windSpeed" => 14.96,"humidity" => 0.86,"dewPoint" => 68,"uvIndex" => 0,"temperature" => 72.49,"windBearing" => 240…)
 Dict{String,Any}("visibility" => 6.216,"apparentTemperature" => 73.48,"time" => 1267210800,"pressure" => 1013,"windSpeed" => 14.96,"humidity" => 0.86,"dewPoint" => 68,"uvIndex" => 0,"temperature" => 72.49,"windBearing" => 240…)

In [36]:
temperature_info_json["hourly"]["data"][:1]

Dict{String,Any} with 10 entries:
  "visibility"          => 6.216
  "apparentTemperature" => 73.48
  "time"                => 1267207200
  "pressure"            => 1013
  "windSpeed"           => 14.96
  "humidity"            => 0.86
  "dewPoint"            => 68
  "uvIndex"             => 0
  "temperature"         => 72.49
  "windBearing"         => 240

In [37]:
temperature_info_json["hourly"]["data"][:2]

Dict{String,Any} with 10 entries:
  "visibility"          => 6.216
  "apparentTemperature" => 73.48
  "time"                => 1267210800
  "pressure"            => 1013
  "windSpeed"           => 14.96
  "humidity"            => 0.86
  "dewPoint"            => 68
  "uvIndex"             => 0
  "temperature"         => 72.49
  "windBearing"         => 240

In [38]:
temperature_info_json["hourly"]["data"][:2]["temperature"]

72.49

## Lets create historical temperature dataset per earthquake

In [41]:
# Create a directory to store the data, if the directory exist it won't overwrite it.
if isdir("../temperature_data")
    println("temperature_data directory exists! \n")
else
    mkdir("../temperature_data")
end

temperature_data directory exists! 



In [92]:
function create_temperature_data(latitude, longitude, timestamp)
    
    temperature_df = DataFrame(date = ["", ""], temperature = [0.0, Missing])
    dates_array = generate_historical_dates(timestamp)
    
    for date in dates_array
        temperature_data = get_historical_weather(latitude, longitude, date)
        println(temperature_data)
        temperature_data_json = JSON.parse(temperature_data)
        
        if haskey(temperature_data_json, "hourly")
            push!(temperature_df, (date = date,
                                   temperature = temperature_data_json["hourly"]["data"][:1]["temperature"]))
        else
            push!(temperature_df, (date = date,
                                   temperature = missing))
        end
        
    end
    
    return temperature_df
    
end
        

create_temperature_data (generic function with 1 method)

## Test the function to create the temperature data

In [93]:
test_df = create_temperature_data(25.93, 128.45, "2010-02-26T20:31:26.970Z")

{"latitude":25.93,"longitude":128.45,"timezone":"Asia/Tokyo","currently":{"time":1267216287,"uvIndex":0},"hourly":{"data":[{"time":1267207200,"temperature":72.49,"apparentTemperature":73.48,"dewPoint":68,"humidity":0.86,"pressure":1013,"windSpeed":14.96,"windBearing":240,"uvIndex":0,"visibility":6.216},{"time":1267210800,"temperature":72.49,"apparentTemperature":73.48,"dewPoint":68,"humidity":0.86,"pressure":1013,"windSpeed":14.96,"windBearing":240,"uvIndex":0,"visibility":6.216}]},"flags":{"sources":["cmc","gfs","icon","isd","madis"],"nearest-station":15.036,"units":"us"},"offset":9}

{"latitude":25.93,"longitude":128.45,"timezone":"Asia/Tokyo","currently":{"time":1267129887,"uvIndex":0},"flags":{"sources":["cmc","gfs","icon","isd","madis"],"nearest-station":49.31,"units":"us"},"offset":9}

{"latitude":25.93,"longitude":128.45,"timezone":"Asia/Tokyo","currently":{"time":1267043487,"uvIndex":0},"flags":{"sources":["cmc","gfs","icon","isd","madis"],"nearest-station":49.31,"units":"us"},

KeyError: KeyError: key "temperature" not found

In [ ]:
#= Once this is tested and works
create function to create all temperatures datasets,
create function to save all datasets to the temperature_data directory
Fix the 
=# 

In [58]:
h = generate_historical_dates("2010-02-26T20:31:26.970Z")

455-element Array{Any,1}:
 "2010-02-26T20:31:26.97"
 "2010-02-25T20:31:26.97"
 "2010-02-24T20:31:26.97"
 "2010-02-23T20:31:26.97"
 "2010-02-22T20:31:26.97"
 "2010-02-21T20:31:26.97"
 "2010-02-20T20:31:26.97"
 "2010-02-19T20:31:26.97"
 "2010-02-18T20:31:26.97"
 "2010-02-17T20:31:26.97"
 "2010-02-16T20:31:26.97"
 "2010-02-15T20:31:26.97"
 "2010-02-14T20:31:26.97"
 ⋮
 "2005-12-09T20:31:26.97"
 "2005-12-08T20:31:26.97"
 "2005-12-07T20:31:26.97"
 "2005-12-06T20:31:26.97"
 "2005-12-05T20:31:26.97"
 "2005-12-04T20:31:26.97"
 "2005-12-03T20:31:26.97"
 "2005-12-02T20:31:26.97"
 "2005-12-01T20:31:26.97"
 "2005-11-30T20:31:26.97"
 "2005-11-29T20:31:26.97"
 "2005-11-28T20:31:26.97"